##雲端硬碟設定指向

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls 'drive'

'My Drive'  'Shared drives'


In [ ]:
import os
path = "/content/drive/My Drive/Colab Notebooks/Bert/AML/"
os.chdir(path)

In [ ]:
!ls

 albert      trained_model   '名字分類(變成姓名學了).ipynb'
 data	     trained_model2  '名字預測(變成姓名學了).ipynb'
 data.zip    trained_model4   新聞分類是否為洗錢新聞.ipynb
 NER.ipynb   分類250.ipynb


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

##下載Albert並放在albert目錄

In [ ]:
!git clone https://github.com/harry83528/albert-zh-for-pytorch-transformers.git albert

fatal: destination path 'albert' already exists and is not an empty directory.


##建立訓練完模型儲存的資料夾

In [ ]:
!mkdir trained_model5

In [ ]:
import torch
from torch.utils.data import TensorDataset
import pickle

In [ ]:
import sys 
sys.path.append('.')
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '.']

In [ ]:
# from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForSequenceClassification

## 定義函式-選擇模型並加載設定

In [ ]:
def use_model(model_name, config_file_path, model_file_path, vocab_file_path, num_labels):
    # 選擇模型並加載設定
    if(model_name == 'bert'):
        from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
        model_config, model_class, model_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
        tokenizer = model_tokenizer(vocab_file=vocab_file_path)
        return model, tokenizer
    elif(model_name == 'albert'):
        from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForSequenceClassification
        model_config, model_class, model_tokenizer = (AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, config=config)
        tokenizer = model_tokenizer.from_pretrained(vocab_file_path)
        return model, tokenizer

## 定義函式-準確率計算

In [ ]:
def compute_accuracy(y_pred, y_target):
    # 計算正確率
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

## 定義函式-快速建立BERT INPUT( tokenizer.build_inputs_with_special_tokens() )

In [ ]:
def to_bert_ids(tokenizer,q_input):
    # 將文字輸入轉換成對應的id編號
    #快速建立BERT INPUT
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q_input)))

## 定義函式-轉換成tensor格式,並且建立dataset

In [ ]:
#轉換成tensor格式,並且建立dataset
def make_dataset(input_ids, input_masks, input_segment_ids, answer_lables):
    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_input_masks = torch.tensor([input_mask for input_mask in input_masks], dtype=torch.long)
    all_input_segment_ids = torch.tensor([input_segment_id for input_segment_id in input_segment_ids], dtype=torch.long)
    all_answer_lables = torch.tensor([answer_lable for answer_lable in answer_lables], dtype=torch.long)    
    return TensorDataset(all_input_ids, all_input_masks, all_input_segment_ids, all_answer_lables)

## 定義函式-分割資料

In [ ]:
def split_dataset(full_dataset, split_rate=0.8):  
    train_size = int(split_rate * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    return train_dataset,test_dataset

In [ ]:
class DataDic(object):
    def __init__(self, answers):
        self.answers = answers #全部答案(含重複)
        self.answers_norepeat = sorted(list(set(answers))) # 不重複
        self.answers_types = len(self.answers_norepeat) # 總共多少類
        self.ans_list = [] # 用於查找id或是text的list
        self._make_dic() # 製作字典
    
    def _make_dic(self):
        for index_a,a in enumerate(self.answers_norepeat):
            if a != None:
                self.ans_list.append((index_a,a))

    def to_id(self,text):
        for ans_id,ans_text in self.ans_list:
            if text == ans_text:
                return ans_id

    def to_text(self,id):
        for ans_id,ans_text in self.ans_list:
            if id == ans_id:
                return ans_text

    @property
    def types(self):
        return self.answers_types
    
    @property
    def data(self):
        return self.answers

    def __len__(self):
        return len(self.answers)

## 定義函式-創建BERT INPUT三層(Token Embeddings、Segment Embeddings、Position Embeddings)

In [ ]:
import json
#創建BERT INPUT的三層
def convert_data_to_feature(tokenizer, train_data_path):
    with open(train_data_path,'r',encoding="utf-8") as load_f:
      load_list = json.load(load_f)

    questions = []
    answers = []
    for item in load_list:
      question_word_piece_list=tokenizer.tokenize(item['context'])
      if len(question_word_piece_list)<511:
        questions.append(item['context'])
      else:
        questions.append(item['context'][:510])

      if item['name'] != '[]':
        answers.append('1')
      else:
        answers.append('0')

    assert len(answers) == len(questions)
    print('questions len',len(questions))
    
    ans_dic = DataDic(answers)
    question_dic = DataDic(questions)

    q_tokens = []
    max_seq_len = 0
    for q in question_dic.data:
        bert_ids = to_bert_ids(tokenizer,q)
        if(len(bert_ids)>max_seq_len):
            max_seq_len = len(bert_ids)
        q_tokens.append(bert_ids)
        #反向查詢(還原)
        # print(tokenizer.convert_ids_to_tokens(tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q)))))
    
    print("最長問句長度:",max_seq_len)
    assert max_seq_len <= 512 # 小於BERT-base長度限制

    # 補齊長度
    for q in q_tokens:
        while len(q)<max_seq_len:
            q.append(0)
    
    a_labels = []
    for a in ans_dic.data:
        a_labels.append(ans_dic.to_id(a))
        # print (ans_dic.to_id(a))
    
    # BERT input embedding
    answer_lables = a_labels
    input_ids = q_tokens
    input_masks = [[1]*max_seq_len for i in range(len(question_dic))]
    input_segment_ids = [[0]*max_seq_len for i in range(len(question_dic))]
    assert len(input_ids) == len(question_dic) and len(input_ids) == len(input_masks) and len(input_ids) == len(input_segment_ids)

    data_features = {'input_ids':input_ids,
                    'input_masks':input_masks,
                    'input_segment_ids':input_segment_ids,
                    'answer_lables':answer_lables,
                    'question_dic':question_dic,
                    'answer_dic':ans_dic}
    
    output = open('trained_model5/data_features.pkl', 'wb')
    pickle.dump(data_features,output)
    return data_features

In [ ]:
!pip install transformers==2.3.0
from torch.utils.data import DataLoader
import torch
from transformers import AdamW

     |████████████████████████████████| 450kB 8.5MB/s 
     |████████████████████████████████| 890kB 18.7MB/s 
     |████████████████████████████████| 1.1MB 38.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5816583548c4e9cc7ab7192677a9e950c995dbbf4800fda833f62ee5006bcf3a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


##主訓練架構

In [ ]:
if __name__ == "__main__":

    #Step1:初始化加載模型 Start==================================================
    # BERT
#     model_setting = {
#         "model_name":"bert", 
#         "config_file_path":"bert-base-chinese", 
#         "model_file_path":"bert-base-chinese", 
#         "vocab_file_path":"bert-base-chinese-vocab.txt",
#         "num_labels":149  # 分幾類 
#     }    


    # ALBERT
    model_setting = {
        "model_name":"albert", 
        "config_file_path":"albert/albert_tiny/config.json", 
        "model_file_path":"albert/albert_tiny/pytorch_model.bin", 
        "vocab_file_path":"albert/albert_tiny/vocab.txt",
        "num_labels":2 # 分幾類
    }    

    #初始化加載模型
    model, tokenizer = use_model(**model_setting)
    #Step1:初始化加載模型 END====================================================


    #Step2:指定硬體裝置 Start====================================================
    # setting device
    #你電腦的 GPU 能否被 PyTorch 調用,如果不行就使用CPU 
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("using device",device)
    model.to(device)
    #Step2:指定硬體裝置 END======================================================

    #Step3:將訓練資料讀入並且組建BERT輸入格式 Start===============================
    #呼叫寫好的convert_data_to_feature創建BERT INPUT的三層
    data_feature = convert_data_to_feature(tokenizer,'data/AML_NER.json')
    input_ids = data_feature['input_ids']
    input_masks = data_feature['input_masks']
    input_segment_ids = data_feature['input_segment_ids']
    answer_lables = data_feature['answer_lables']
    #Step3:將訓練資料讀入並且組建BERT輸入格式 END=================================


    #Step4:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader Srart==
    full_dataset = make_dataset(input_ids = input_ids, input_masks = input_masks, input_segment_ids = input_segment_ids, answer_lables = answer_lables)
    train_dataset, test_dataset = split_dataset(full_dataset, 0.9)
    train_dataloader = DataLoader(train_dataset,batch_size=16,shuffle=True)
    test_dataloader = DataLoader(test_dataset,batch_size=16,shuffle=True)
     #Step4:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader END===

    #準備優化器 Start===========================================================
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=5e-6, eps=1e-8)
    # scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps, t_total=t_total)
    #準備優化器 End===========================================================


    #Step5:建構training loop、開始訓練 Start=====================================
    model.zero_grad() #梯度歸零
    for epoch in range(50):
        running_loss_val = 0.0
        running_acc = 0.0
        for batch_index, batch_dict in enumerate(train_dataloader):
            model.train()
            batch_dict = tuple(t.to(device) for t in batch_dict)
            outputs = model(
                batch_dict[0],
                # attention_mask=batch_dict[1],
                labels = batch_dict[3]
                )
            loss,logits = outputs[:2]
            loss.sum().backward()
            optimizer.step()
            # scheduler.step()  # Update learning rate schedule
            model.zero_grad()
            
            # compute the loss
            loss_t = loss.item()
            running_loss_val += (loss_t - running_loss_val) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(logits, batch_dict[3])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # log
            print("epoch:%2d batch:%4d train_loss:%2.4f train_acc:%3.4f"%(epoch+1, batch_index+1, running_loss_val, running_acc))
        
        running_loss_val = 0.0
        running_acc = 0.0
        for batch_index, batch_dict in enumerate(test_dataloader):
            model.eval()
            batch_dict = tuple(t.to(device) for t in batch_dict)
            outputs = model(
                batch_dict[0],
                # attention_mask=batch_dict[1],
                labels = batch_dict[3]
                )
            loss,logits = outputs[:2]
            
            # compute the loss
            loss_t = loss.item()
            running_loss_val += (loss_t - running_loss_val) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(logits, batch_dict[3])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # log
            print("epoch:%2d batch:%4d test_loss:%2.4f test_acc:%3.4f"%(epoch+1, batch_index+1, running_loss_val, running_acc))
    
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained('trained_model5')
    #Step5:建構training loop、開始訓練 END=====================================

using device cuda
questions len 5023
最長問句長度: 512


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Streaming output truncated to the last 5000 lines.
epoch:35 batch:  42 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  43 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  44 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  45 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  46 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  47 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  48 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  49 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  50 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  51 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  52 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  53 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  54 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  55 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  56 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  57 train_loss:0.0001 train_acc:100.0000
epoch:35 batch:  58 train_loss:0.0001

In [ ]:
!pip install -U ckiptagger[tfgpu,gdown]

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [ ]:
def get_person_str(context):
  # 使用 GPU：
  #    1. 安裝 tensorflow-gpu (請見安裝說明)
  #    2. 設定 CUDA_VISIBLE_DEVICES 環境變數，例如：os.environ["CUDA_VISIBLE_DEVICES"] = "0"
  #    3. 設定 disable_cuda=False，例如：ws = WS("./data", disable_cuda=False)
  # 使用 CPU：
  ws = WS("./data", disable_cuda=False)
  pos = POS("./data", disable_cuda=False)
  ner = NER("./data", disable_cuda=False)
  sentence_list = [context]

  word_sentence_list = ws(
      sentence_list,
      # sentence_segmentation = True, # To consider delimiters
      # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
      # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
      # coerce_dictionary = dictionary2, # words in this dictionary are forced
  )

  pos_sentence_list = pos(word_sentence_list)
  entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
  #print(entity_sentence_list)

  set_entity=set()    
  for i, sentence in enumerate(sentence_list):
      #print()
      #print(f"'{sentence}'")
      #print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
      for entity in sorted(entity_sentence_list[i]):
            if entity[2] == 'PERSON':
              #print(entity[3])
              set_entity.add(entity[3].replace(" ", ""))
  str_entity = ','.join(set_entity)
  #print(str_entity)
  return str_entity

In [ ]:
# load and init
pkl_file = open('trained_model5/data_features.pkl', 'rb')
data_features = pickle.load(pkl_file)
answer_dic = data_features['answer_dic']

In [ ]:
model_setting2 = {
    "model_name":"albert", 
    "config_file_path":"trained_model5/config.json", 
    "model_file_path":"trained_model5/pytorch_model.bin", 
    "vocab_file_path":"albert/albert_tiny/vocab.txt",
    "num_labels":2 # 分幾類
}    
model, tokenizer = use_model(**model_setting2)
model.eval()

#
q_inputs = ['香港海關本周三(23日)於機場入境大堂 ，例行檢查2名從秘魯利馬出發，經法國巴黎轉機抵港的男女時，在2人的行李篋內共發現10支以泡泡紙及膠紙包裹的紅酒，初步調查後證實檢獲的10支液態可卡因共重約22公斤，市值約2300萬港元。2名被控販毒罪的男女今早(26日)被解往西九龍裁判法院提堂，暫時毋須答辯，控方要求將案押後至今年12月23日再提堂，期間對涉案懷疑毒品作化驗，2人續還押監房看管。												本案兩名被告分別是28歲女子沈陳婕楹及39歲男子沈駿州，均持台灣護照入境，在港無居所，庭上無提及兩人的關係。他們共同被控一項販毒罪，控罪指兩人在今年10月23日，在香港國際機場一號客運大樓海關 無為兩人申請保釋。 ']

for q_input in q_inputs:
    old_q_input=q_input
    if len(tokenizer.tokenize(q_input)) >511 :
      q_input=q_input[0:510]
    bert_ids = to_bert_ids(tokenizer,q_input)
    print('len(bert_ids)',len(bert_ids))
    assert len(bert_ids) <= 512
    input_ids = torch.LongTensor(bert_ids).unsqueeze(0)

    # predict
    outputs = model(input_ids)
    predicts = outputs[:2]
    predicts = predicts[0]
    max_val = torch.max(predicts)
    label = (predicts == max_val).nonzero().numpy()[0][1]
    ans_label = answer_dic.to_text(label)
    
    print(q_input)
    print(ans_label)
    if ans_label='0':
        print('[]');
    else:
        print(get_person_str(old_q_input))

len(bert_ids) 286
香港海關本周三(23日)於機場入境大堂 ，例行檢查2名從秘魯利馬出發，經法國巴黎轉機抵港的男女時，在2人的行李篋內共發現10支以泡泡紙及膠紙包裹的紅酒，初步調查後證實檢獲的10支液態可卡因共重約22公斤，市值約2300萬港元。2名被控販毒罪的男女今早(26日)被解往西九龍裁判法院提堂，暫時毋須答辯，控方要求將案押後至今年12月23日再提堂，期間對涉案懷疑毒品作化驗，2人續還押監房看管。												本案兩名被告分別是28歲女子沈陳婕楹及39歲男子沈駿州，均持台灣護照入境，在港無居所，庭上無提及兩人的關係。他們共同被控一項販毒罪，控罪指兩人在今年10月23日，在香港國際機場一號客運大樓海關 無為兩人申請保釋。 
1

